In [67]:
import pandas as pd
import requests as r
import json

In [68]:
#setting konfigurasi
with open("../env.json", "r", encoding="utf-8") as file:
    config = json.load(file)  # Membaca dan parsing JSON
varId = 1

## Fungsi Utama

In [69]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [70]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId=varId):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [71]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId=varId,domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

In [90]:
def clean_tag_html(data):
    data.loc[:, "vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)
    return data
    

In [72]:
daftar_data=getVarByDomain(domain='5100')

### (1) Inflasi Tahunan Kota Denpasar, Singaraja, dan Nasional

In [82]:
data1=getDataByVarId(domain='5100',varId=varId)
data1.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67


In [83]:
#proses cleaning
data1=data1[['verval_val','vervar_label','tahun_label','content_label']]
#proses rename colomn
data1_tb=data1.rename(columns={'vervar_label' : 'wilayah_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data1_tb=data1_tb.pivot_table(index=['verval_val','wilayah_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data1_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data1_tb.columns.values]
data1_tb=data1_tb.reset_index()
data1_tb.drop(columns=['verval_val'])
data1_tb.head()

,verval_val,wilayah_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,3,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,4,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,5,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67


In [75]:
# Export to Excel
data1_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (47) Inflasi Tahunan Kota Singaraja Menurut 7 Kelompok Pengeluaran

In [84]:
varId = 47
data47=getDataByVarId(domain='5100',varId=varId)
data47.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,47,0,Tidak ada,119,2019,0,Tahun,14701190,14701190,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",47,0,Tidak ada,119,2019,0,Tahun,24701190,24701190,3.82
2,3,Perumahan,47,0,Tidak ada,119,2019,0,Tahun,34701190,34701190,0.60
3,4,Sandang,47,0,Tidak ada,119,2019,0,Tahun,44701190,44701190,4.02
4,5,Kesehatan,47,0,Tidak ada,119,2019,0,Tahun,54701190,54701190,5.67


In [85]:
#proses cleaning
data47=data47[['verval_val','vervar_label','tahun_label','content_label']]
data47 = clean_tag_html(data47)
#proses rename colomn
data47_tb=data47.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data47_tb=data47_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data47_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data47_tb.columns.values]
data47_tb=data47_tb.reset_index()
data47_tb.drop(columns=['verval_val'])
data47_tb.head()

,verval_val,kelompok_inflasi,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,8.09,4.74,9.98,-3.39,4.33,1.97
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",15.00,4.22,4.08,3.15,0.77,3.82
2,3,Perumahan,9.97,3.33,1.83,8.44,-3.07,0.60
3,4,Sandang,6.25,7.73,8.47,6.53,4.42,4.02
4,5,Kesehatan,1.45,2.61,4.65,5.02,2.67,5.67


In [86]:
# Export to Excel
data47_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (65) Inflasi Tahunan Kota Denpasar Menurut 7 Kelompok Pengeluaran

In [91]:
varId = 65
data65=getDataByVarId(domain='5100',varId=varId)
data65.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Bahan Makanan,65,0,Tidak ada,119,2019,0,Tahun,16501190,16501190,0.76
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",65,0,Tidak ada,119,2019,0,Tahun,26501190,26501190,5.98
2,3,Perumahan,65,0,Tidak ada,119,2019,0,Tahun,36501190,36501190,1.34
3,4,Sandang,65,0,Tidak ada,119,2019,0,Tahun,46501190,46501190,2.22
4,5,Kesehatan,65,0,Tidak ada,119,2019,0,Tahun,56501190,56501190,3.16


In [ ]:
#proses cleaning
data65=data65[['verval_val','vervar_label','tahun_label','content_label']]
data65 = clean_tag_html(data65)
#proses rename colomn
data65_tb=data65.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data65_tb=data65_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data65_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data65_tb.columns.values]
data65_tb=data65_tb.reset_index()
data65_tb.head()

,verval_val,kelompok_inflasi,inflasi_2010,inflasi_2011,inflasi_2012,inflasi_2013,inflasi_2014,inflasi_2015,inflasi_2016,inflasi_2017,inflasi_2018,inflasi_2019
0,1,Bahan Makanan,18.32,3.49,6.06,10.36,11.23,3.31,4.51,1.96,5.86,0.76
1,2,"Makanan Jadi, Minuman, Rokok, dan Tembakau",10.20,5.92,8.67,5.19,6.29,3.55,7.70,6.85,2.39,5.98
2,3,Perumahan,7.31,4.13,3.97,5.64,7.20,5.09,0.39,2.45,1.56,1.34
3,4,Sandang,2.92,6.23,0.92,-1.24,3.60,3.61,5.41,0.32,1.47,2.22
4,5,Kesehatan,1.13,2.41,2.13,1.49,9.99,5.26,3.68,1.48,2.69,3.16


In [93]:
# Export to Excel
data65_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (314) Inflasi Bulanan Kota Denpasar, Singaraja, dan Nasional

In [105]:
varId = 314
data314=getDataByVarId(domain='5100',varId=varId)
data314.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Denpasar,314,0,Tidak ada,123,2023,1,Januari,131401231,131401231,0.62
1,1,Denpasar,314,0,Tidak ada,123,2023,2,Februari,131401232,131401232,0.04
2,1,Denpasar,314,0,Tidak ada,123,2023,3,Maret,131401233,131401233,0.03
3,1,Denpasar,314,0,Tidak ada,123,2023,4,April,131401234,131401234,0.06
4,1,Denpasar,314,0,Tidak ada,123,2023,5,Mei,131401235,131401235,0.34


In [122]:
# proses cleaning
data314 = data314[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data314_tb = data314.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'inflasi',
    'turtahun_label':'bulan'
})

data314_tb["periode"] = data314_tb["tahun"].astype(str) + "_" + data314_tb["bulan"].astype(str)
data314_tb = data314_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['inflasi']
).reset_index()

data314_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data314_tb.columns.values
]
data314_tb.head()

,verval_val,kota_inflasi,inflasi_2014_Agustus,inflasi_2014_April,inflasi_2014_Desember,inflasi_2014_Februari,inflasi_2014_Januari,inflasi_2014_Juli,inflasi_2014_Juni,inflasi_2014_Maret,...,inflasi_2023_Februari,inflasi_2023_Januari,inflasi_2023_Juli,inflasi_2023_Juni,inflasi_2023_Maret,inflasi_2023_Mei,inflasi_2023_November,inflasi_2023_Oktober,inflasi_2023_September,inflasi_2023_Tahunan
0,1,Denpasar,0.66,0.13,1.99,0.37,1.26,0.49,-0.20,0.32,...,0.04,0.62,0.36,-0.08,0.03,0.34,0.33,0.15,-0.03,2.54
1,2,Singaraja,0.77,0.13,2.80,0.37,0.83,0.39,-0.61,0.17,...,0.29,0.95,0.25,0.22,0.42,0.37,0.87,0.44,-0.05,4.31
2,3,Nasional,0.47,-0.02,2.46,0.26,1.07,0.93,0.43,0.08,...,0.16,0.34,0.21,0.14,0.18,0.09,0.38,0.17,0.19,2.61


In [123]:
data314_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (362) Indeks Harga Konsumen (IHK) Bulanan Kota Denpasar, Singaraja, dan Nasional

In [124]:
varId = 362
data362=getDataByVarId(domain='5100',varId=varId)
data362.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Denpasar,362,0,Tidak ada,123,2023,1,Januari,136201231,136201231,113.87
1,1,Denpasar,362,0,Tidak ada,123,2023,2,Februari,136201232,136201232,113.91
2,1,Denpasar,362,0,Tidak ada,123,2023,3,Maret,136201233,136201233,113.94
3,1,Denpasar,362,0,Tidak ada,123,2023,4,April,136201234,136201234,114.01
4,1,Denpasar,362,0,Tidak ada,123,2023,5,Mei,136201235,136201235,114.40


In [126]:
# proses cleaning
data362 = data362[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data362_tb = data362.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'IHK',
    'turtahun_label':'bulan'
})

data362_tb["periode"] = data362_tb["tahun"].astype(str) + "_" + data362_tb["bulan"].astype(str)
data362_tb = data362_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['IHK']
).reset_index()

data362_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data362_tb.columns.values
]
data362_tb.head()

,verval_val,kota_inflasi,IHK_2014_Agustus,IHK_2014_April,IHK_2014_Desember,IHK_2014_Februari,IHK_2014_Januari,IHK_2014_Juli,IHK_2014_Juni,IHK_2014_Maret,...,IHK_2023_Desember,IHK_2023_Februari,IHK_2023_Januari,IHK_2023_Juli,IHK_2023_Juni,IHK_2023_Maret,IHK_2023_Mei,IHK_2023_November,IHK_2023_Oktober,IHK_2023_September
0,1,Denpasar,111.42,110.03,116.44,109.54,109.14,110.69,110.15,109.89,...,116.05,113.91,113.87,114.72,114.31,113.94,114.40,115.48,115.10,114.93
1,2,Singaraja,117.70,115.49,125.47,115.10,114.67,116.80,116.35,115.30,...,118.67,115.18,114.85,116.38,116.09,115.66,115.84,118.16,117.14,116.63
2,3,Nasional,113.58,111.35,119.00,111.28,110.99,113.05,112.01,111.37,...,116.56,114.16,113.98,115.24,115.00,114.36,114.84,116.68,115.64,115.44


In [127]:
data362_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (374) Inflasi Tahunan Kota Singaraja Menurut 11 Kelompok Pengeluaran

In [133]:
varId = 374
data374=getDataByVarId(domain='5100',varId=varId)
data374.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Makanan, Minuman, dan Tembakau",374,0,Tidak ada,123,2023,0,Tahun,137401230,137401230,10.12
1,2,Pakaian dan Alas Kaki,374,0,Tidak ada,123,2023,0,Tahun,237401230,237401230,1.25
2,3,"Perumahan, Air, Listrik, dan Bahan Bakar Rumah...",374,0,Tidak ada,123,2023,0,Tahun,337401230,337401230,2.58
3,4,"Perlengkapan, Peralatan, dan Pemeliharaan Ruti...",374,0,Tidak ada,123,2023,0,Tahun,437401230,437401230,-3.28
4,5,Kesehatan,374,0,Tidak ada,123,2023,0,Tahun,537401230,537401230,0.41


In [134]:
#proses cleaning
data374=data374[['verval_val','vervar_label','tahun_label','content_label']]
data374 = clean_tag_html(data374)
#proses rename colomn
data374_tb=data374.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data374_tb=data374_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data374_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data374_tb.columns.values]
data374_tb=data374_tb.reset_index()
data374_tb.drop(columns=['verval_val'])
data374_tb.head()

,verval_val,kelompok_inflasi,inflasi_2020,inflasi_2021,inflasi_2022,inflasi_2023
0,1,"Makanan, Minuman, dan Tembakau",5.90,1.46,6.22,10.12
1,2,Pakaian dan Alas Kaki,3.52,1.41,0.73,1.25
2,3,"Perumahan, Air, Listrik, dan Bahan Bakar Rumah...",0.31,0.17,3.06,2.58
3,4,"Perlengkapan, Peralatan, dan Pemeliharaan Ruti...",-3.57,17.28,-1.97,-3.28
4,5,Kesehatan,2.85,2.42,0.56,0.41


In [135]:
data374_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (375) Inflasi Tahunan Kota Denpasar Menurut 11 Kelompok Pengeluaran

In [128]:
varId = 375
data375=getDataByVarId(domain='5100',varId=varId)
data375.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Makanan, Minuman, dan Tembakau",375,0,Tidak ada,123,2023,0,Tahun,137501230,137501230,6.36
1,2,Pakaian dan Alas Kaki,375,0,Tidak ada,123,2023,0,Tahun,237501230,237501230,-1.61
2,3,"Perumahan, Air, Listrik, dan Bahan Bakar Rumah...",375,0,Tidak ada,123,2023,0,Tahun,337501230,337501230,0.93
3,4,"Perlengkapan, Peralatan, dan Pemeliharaan Ruti...",375,0,Tidak ada,123,2023,0,Tahun,437501230,437501230,-0.46
4,5,Kesehatan,375,0,Tidak ada,123,2023,0,Tahun,537501230,537501230,0.82


In [130]:
#proses cleaning
data375=data375[['verval_val','vervar_label','tahun_label','content_label']]
data375 = clean_tag_html(data375)
#proses rename colomn
data375_tb=data375.rename(columns={'vervar_label' : 'kelompok_inflasi','tahun_label' : 'tahun','content_label' : 'inflasi'})
#pivoting tabel
data375_tb=data375_tb.pivot_table(index=['verval_val','kelompok_inflasi'],columns=['tahun'],values=['inflasi'])
#multi index jadikan 1
data375_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data375_tb.columns.values]
data375_tb=data375_tb.reset_index()
data375_tb.drop(columns=['verval_val'])
data375_tb.head()

,verval_val,kelompok_inflasi,inflasi_2020,inflasi_2021,inflasi_2022,inflasi_2023
0,1,"Makanan, Minuman, dan Tembakau",1.55,3.95,6.70,6.36
1,2,Pakaian dan Alas Kaki,-0.68,-2.50,1.18,-1.61
2,3,"Perumahan, Air, Listrik, dan Bahan Bakar Rumah...",-0.32,-0.02,4.60,0.93
3,4,"Perlengkapan, Peralatan, dan Pemeliharaan Ruti...",1.06,10.39,8.63,-0.46
4,5,Kesehatan,1.66,0.78,2.78,0.82


In [131]:
data375_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (402) Inflasi Bulanan (month to month) Provinsi Bali

In [136]:
varId = 402
data402=getDataByVarId(domain='5100',varId=varId)
data402.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Tabanan,402,0,Tidak ada,125,2025,1,Januari,140201251,140201251,0.48
1,1,Kabupaten Tabanan,402,0,Tidak ada,125,2025,2,Februari,140201252,140201252,-1.05
2,1,Kabupaten Tabanan,402,0,Tidak ada,125,2025,3,Maret,140201253,140201253,1.52
3,1,Kabupaten Tabanan,402,0,Tidak ada,125,2025,4,April,140201254,140201254,1.09
4,1,Kabupaten Tabanan,402,0,Tidak ada,125,2025,5,Mei,140201255,140201255,-0.82


In [137]:
# proses cleaning
data402 = data402[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data402_tb = data402.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'inflasi',
    'turtahun_label':'bulan'
})

data402_tb["periode"] = data402_tb["tahun"].astype(str) + "_" + data402_tb["bulan"].astype(str)
data402_tb = data402_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['inflasi']
).reset_index()

data402_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data402_tb.columns.values
]
data402_tb.head()

,verval_val,kota_inflasi,inflasi_2024_Agustus,inflasi_2024_April,inflasi_2024_Desember,inflasi_2024_Februari,inflasi_2024_Januari,inflasi_2024_Juli,inflasi_2024_Juni,inflasi_2024_Maret,...,inflasi_2024_Oktober,inflasi_2024_September,inflasi_2024_Tahunan,inflasi_2025_April,inflasi_2025_Februari,inflasi_2025_Januari,inflasi_2025_Juli,inflasi_2025_Juni,inflasi_2025_Maret,inflasi_2025_Mei
0,1,Kabupaten Tabanan,0.28,0.42,0.49,0.68,-0.07,0.09,-1.09,0.91,...,-0.03,0.26,2.44,1.09,-1.05,0.48,0.30,0.29,1.52,-0.82
1,2,Kabupaten Badung,-0.09,0.03,0.37,0.58,-0.01,-0.03,-0.63,1.10,...,-0.02,0.09,1.98,0.49,-0.89,0.47,0.15,0.53,1.45,-0.95
2,3,Singaraja,-0.18,0.07,0.32,0.51,-0.22,0.12,-0.53,0.89,...,0.21,0.25,1.93,0.80,-0.81,-0.53,0.42,0.37,1.71,-0.28
3,4,Kota Denpasar,0.26,0.53,0.19,0.65,-0.08,0.16,-0.32,0.87,...,0.10,0.06,2.69,0.69,-0.13,-0.27,0.37,0.48,1.69,-0.16
4,5,Provinsi Bali,0.10,0.32,0.31,0.61,-0.09,0.10,-0.55,0.93,...,0.07,0.13,2.34,0.73,-0.57,-0.02,0.32,0.44,1.61,-0.47


In [138]:
data402_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (405) Indeks Harga Konsumen (IHK) Bulanan Provinsi Bali

In [139]:
varId = 405
data405=getDataByVarId(domain='5100',varId=varId)
data405.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Tabanan,405,0,Tidak ada,125,2025,1,Januari,140501251,140501251,110.72
1,1,Kabupaten Tabanan,405,0,Tidak ada,125,2025,2,Februari,140501252,140501252,109.56
2,1,Kabupaten Tabanan,405,0,Tidak ada,125,2025,3,Maret,140501253,140501253,111.22
3,1,Kabupaten Tabanan,405,0,Tidak ada,125,2025,4,April,140501254,140501254,112.43
4,1,Kabupaten Tabanan,405,0,Tidak ada,125,2025,5,Mei,140501255,140501255,111.51


In [140]:
# proses cleaning
data405 = data405[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data405_tb = data405.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'IHK',
    'turtahun_label':'bulan'
})

data405_tb["periode"] = data405_tb["tahun"].astype(str) + "_" + data405_tb["bulan"].astype(str)
data405_tb = data405_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['IHK']
).reset_index()

data405_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data405_tb.columns.values
]
data405_tb.head()

,verval_val,kota_inflasi,IHK_2024_Agustus,IHK_2024_April,IHK_2024_Desember,IHK_2024_Februari,IHK_2024_Januari,IHK_2024_Juli,IHK_2024_Juni,IHK_2024_Maret,...,IHK_2024_November,IHK_2024_Oktober,IHK_2024_September,IHK_2025_April,IHK_2025_Februari,IHK_2025_Januari,IHK_2025_Juli,IHK_2025_Juni,IHK_2025_Maret,IHK_2025_Mei
0,1,Kabupaten Tabanan,108.57,109.67,110.19,108.23,107.50,108.27,108.17,109.21,...,109.65,108.82,108.85,112.43,109.56,110.72,112.17,111.83,111.22,111.51
1,2,Kabupaten Badung,104.85,105.73,106.02,104.55,103.95,104.94,104.97,105.70,...,105.63,104.92,104.94,107.63,105.57,106.52,107.34,107.18,107.10,106.61
2,3,Singaraja,106.02,106.99,107.70,105.97,105.43,106.21,106.08,106.91,...,107.36,106.50,106.28,108.94,106.26,107.13,109.50,109.04,108.08,108.64
3,4,Kota Denpasar,107.58,107.42,108.16,105.93,105.25,107.30,107.13,106.85,...,107.95,107.75,107.64,110.31,107.73,107.87,111.07,110.66,109.55,110.13
4,5,Provinsi Bali,106.80,107.28,107.87,105.95,105.31,106.69,106.58,106.94,...,107.54,107.01,106.94,109.75,107.23,107.85,110.06,109.71,108.96,109.23


In [141]:
data405_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (407) Inflasi y-o-y (year on year) Provinsi Bali

In [142]:
varId = 407
data407=getDataByVarId(domain='5100',varId=varId)
data407.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Tabanan,407,0,Tidak ada,125,2025,1,Januari,140701251,140701251,3.00
1,1,Kabupaten Tabanan,407,0,Tidak ada,125,2025,2,Februari,140701252,140701252,1.23
2,1,Kabupaten Tabanan,407,0,Tidak ada,125,2025,3,Maret,140701253,140701253,1.84
3,1,Kabupaten Tabanan,407,0,Tidak ada,125,2025,4,April,140701254,140701254,2.52
4,1,Kabupaten Tabanan,407,0,Tidak ada,125,2025,5,Mei,140701255,140701255,1.97


In [144]:
# proses cleaning
data407 = data407[['verval_val','vervar_label','tahun_label','content_label','turtahun_label']]
data407_tb = data407.rename(columns={
    'vervar_label' : 'kota_inflasi',
    'tahun_label'  : 'tahun',
    'content_label': 'inflasi',
    'turtahun_label':'bulan'
})

data407_tb["periode"] = data407_tb["tahun"].astype(str) + "_" + data407_tb["bulan"].astype(str)
data407_tb = data407_tb.pivot_table(
    index=['verval_val','kota_inflasi'],
    columns=['periode'],
    values=['inflasi']
).reset_index()

data407_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data407_tb.columns.values
]
data407_tb.head()

,verval_val,kota_inflasi,inflasi_2024_Agustus,inflasi_2024_April,inflasi_2024_Desember,inflasi_2024_Februari,inflasi_2024_Januari,inflasi_2024_Juli,inflasi_2024_Juni,inflasi_2024_Maret,...,inflasi_2024_November,inflasi_2024_Oktober,inflasi_2024_September,inflasi_2025_April,inflasi_2025_Februari,inflasi_2025_Januari,inflasi_2025_Juli,inflasi_2025_Juni,inflasi_2025_Maret,inflasi_2025_Mei
0,1,Kabupaten Tabanan,1.68,4.42,2.44,3.87,3.79,1.75,1.96,3.95,...,2.29,2.31,2.98,2.52,1.23,3.00,3.60,3.38,1.84,1.97
1,2,Kabupaten Badung,2.05,4.15,1.98,2.90,2.62,2.40,2.75,3.92,...,2.44,2.40,2.53,1.80,0.98,2.47,2.29,2.11,1.32,0.92
2,3,Singaraja,1.69,3.69,1.93,3.02,2.80,2.07,2.14,3.71,...,1.98,1.71,1.78,1.82,0.27,1.61,3.10,2.79,1.09,1.88
3,4,Kota Denpasar,2.95,3.94,2.69,2.72,2.12,3.04,3.18,3.43,...,2.82,2.96,2.99,2.69,1.70,2.49,3.51,3.30,2.53,2.48
4,5,Provinsi Bali,2.32,4.02,2.34,2.98,2.61,2.53,2.71,3.67,...,2.50,2.51,2.67,2.30,1.21,2.41,3.16,2.94,1.89,1.92


In [145]:
data407_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (319) Indeks Kemahalan Konstruksi Menurut Kabupaten/Kota di Provinsi Bali

In [165]:
varId = 319
data319=getDataByVarId(domain='5100',varId=varId)
data319.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,319,0,Tidak ada,123,2023,0,Tahun,131901230,131901230,100.09
1,2,Kab. Tabanan,319,0,Tidak ada,123,2023,0,Tahun,231901230,231901230,100.45
2,3,Kab. Badung,319,0,Tidak ada,123,2023,0,Tahun,331901230,331901230,105.73
3,4,Kab. Gianyar,319,0,Tidak ada,123,2023,0,Tahun,431901230,431901230,100.22
4,5,Kab. Klungkung,319,0,Tidak ada,123,2023,0,Tahun,531901230,531901230,108.15


In [166]:
#proses cleaning
data319=data319[['verval_val','vervar_label','tahun_label','content_label']]
data319 = clean_tag_html(data319)
#proses rename colomn
data319_tb=data319.rename(columns={'vervar_label' : 'kabupaten_kota','tahun_label' : 'tahun','content_label' : 'IKK'})
#pivoting tabel
data319_tb=data319_tb.pivot_table(index=['verval_val','kabupaten_kota'],columns=['tahun'],values=['IKK'])
#multi index jadikan 1
data319_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data319_tb.columns.values]
data319_tb=data319_tb.reset_index()
data319_tb.drop(columns=['verval_val'])
data319_tb.head()

,verval_val,kabupaten_kota,IKK_2014,IKK_2015,IKK_2016,IKK_2017,IKK_2018,IKK_2019,IKK_2020,IKK_2021,IKK_2022,IKK_2023
0,1,Kab. Jembrana,88.37,107.02,113.47,112.93,124.74,126.59,122.49,103.86,103.04,100.09
1,2,Kab. Tabanan,101.06,105.69,111.79,116.36,126.12,122.39,116.18,99.21,101.12,100.45
2,3,Kab. Badung,91.82,106.08,111.09,114.54,126.35,126.12,121.15,106.58,107.92,105.73
3,4,Kab. Gianyar,81.63,98.73,105.32,112.40,120.51,114.65,105.10,97.44,102.77,100.22
4,5,Kab. Klungkung,83.30,99.65,106.30,101.40,112.33,110.16,111.95,106.57,107.55,108.15


In [167]:
data319_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (380) Banyaknya Perusahaan Konstruksi Aktif Menurut Kabupaten/Kota di Provinsi Bali

In [146]:
varId = 380
data380=getDataByVarId(domain='5100',varId=varId)
data380.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,380,0,Tidak ada,124,2024,0,Tahun,138001240,138001240,149
1,2,Kab. Tabanan,380,0,Tidak ada,124,2024,0,Tahun,238001240,238001240,123
2,3,Kab. Badung,380,0,Tidak ada,124,2024,0,Tahun,338001240,338001240,279
3,4,Kab. Gianyar,380,0,Tidak ada,124,2024,0,Tahun,438001240,438001240,167
4,5,Kab. Klungkung,380,0,Tidak ada,124,2024,0,Tahun,538001240,538001240,70


In [149]:
#proses cleaning
data380=data380[['verval_val','vervar_label','tahun_label','content_label']]
#proses rename colomn
data380 = clean_tag_html(data380)
data380_tb=data380.rename(columns={'vervar_label' : 'kabupaten_kota','tahun_label' : 'tahun','content_label' : 'konstruksi'})
#pivoting tabel
data380_tb=data380_tb.pivot_table(index=['verval_val','kabupaten_kota'],columns=['tahun'],values=['konstruksi'])
#multi index jadikan 1
data380_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data380_tb.columns.values]
data380_tb=data380_tb.reset_index()
data380_tb.drop(columns=['verval_val'])
data380_tb.head()

,verval_val,kabupaten_kota,konstruksi_2017,konstruksi_2018,konstruksi_2019,konstruksi_2020,konstruksi_2021,konstruksi_2022,konstruksi_2023,konstruksi_2024
0,1,Kab. Jembrana,137.0,151.0,143.0,131.0,131.0,158.0,150.0,149.0
1,2,Kab. Tabanan,172.0,130.0,124.0,101.0,95.0,146.0,134.0,123.0
2,3,Kab. Badung,308.0,243.0,267.0,228.0,222.0,265.0,275.0,279.0
3,4,Kab. Gianyar,230.0,207.0,201.0,184.0,183.0,210.0,181.0,167.0
4,5,Kab. Klungkung,202.0,170.0,186.0,156.0,166.0,125.0,97.0,70.0


In [151]:
data380_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (38) Rata-rata Konsumsi per Kapita Sebulan Provinsi Bali Menurut Kabupaten/Kota

In [152]:
varId = 38
data38=getDataByVarId(domain='5100',varId=varId)
data38.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,38,32,Makanan,124,2024,0,Tahun,138321240,138321240,656850
1,1,Kab. Jembrana,38,33,Non Makanan,124,2024,0,Tahun,138331240,138331240,563165
2,1,Kab. Jembrana,38,34,Jumlah,124,2024,0,Tahun,138341240,138341240,1220015
3,2,Kab. Tabanan,38,32,Makanan,124,2024,0,Tahun,238321240,238321240,731545
4,2,Kab. Tabanan,38,33,Non Makanan,124,2024,0,Tahun,238331240,238331240,962778


In [155]:
# proses cleaning
data38 = data38[['verval_val','vervar_label','tahun_label','content_label','turvar_label']]
data38 = clean_tag_html(data38)
data38_tb = data38.rename(columns={
    'vervar_label' : 'kabupaten_kota',
    'tahun_label'  : 'tahun',
    'content_label': 'konsumsi',
    'turvar_label':'jenis'
})

data38_tb["jenis_konsumsi"] = data38_tb["tahun"].astype(str) + "_" + data38_tb["jenis"].astype(str)
data38_tb = data38_tb.pivot_table(
    index=['verval_val','kabupaten_kota'],
    columns=['jenis_konsumsi'],
    values=['konsumsi']
).reset_index()

data38_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data38_tb.columns.values
]
data38_tb.head()

,verval_val,kabupaten_kota,konsumsi_2015_Jumlah,konsumsi_2015_Makanan,konsumsi_2015_Non Makanan,konsumsi_2016_Jumlah,konsumsi_2016_Makanan,konsumsi_2016_Non Makanan,konsumsi_2017_Jumlah,konsumsi_2017_Makanan,...,konsumsi_2021_Non Makanan,konsumsi_2022_Jumlah,konsumsi_2022_Makanan,konsumsi_2022_Non Makanan,konsumsi_2023_Jumlah,konsumsi_2023_Makanan,konsumsi_2023_Non Makanan,konsumsi_2024_Jumlah,konsumsi_2024_Makanan,konsumsi_2024_Non Makanan
0,1,Kab. Jembrana,756299.0,386330.0,369968.0,836284.0,395943.0,440341.0,916813.0,481004.0,...,557105.0,1128905.0,546555.0,582350.0,1213080.0,586854.0,626227.0,1220015.0,656850.0,563165.0
1,2,Kab. Tabanan,1026816.0,435385.0,591431.0,1060742.0,476230.0,584512.0,1099272.0,512611.0,...,709218.0,1095066.0,528160.0,566907.0,1650927.0,696041.0,954886.0,1694323.0,731545.0,962778.0
2,3,Kab. Badung,1192583.0,474871.0,717712.0,1356604.0,552844.0,803760.0,2047529.0,803266.0,...,1081761.0,1741969.0,707274.0,1034694.0,2196165.0,914434.0,1281731.0,2584659.0,1081710.0,1502949.0
3,4,Kab. Gianyar,1108409.0,382592.0,725817.0,1115179.0,432247.0,682932.0,1210671.0,487665.0,...,984078.0,1725527.0,638938.0,1086589.0,1912609.0,709628.0,1202982.0,1940056.0,812387.0,1127669.0
4,5,Kab. Klungkung,731393.0,371234.0,360160.0,769131.0,402681.0,366450.0,801947.0,406485.0,...,484229.0,1283547.0,561560.0,721987.0,1360177.0,635127.0,725050.0,1349130.0,640809.0,708321.0


In [158]:
data38_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (40) Distribusi Pendapatan Provinsi Bali Menurut Kelompok Pendapatan dan Kabupaten/Kota

In [159]:
varId = 40
data40=getDataByVarId(domain='5100',varId=varId)
data40.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,40,35,40% Rendah,124,2024,0,Tahun,140351240,140351240,23.57
1,1,Kab. Jembrana,40,36,40% Sedang,124,2024,0,Tahun,140361240,140361240,36.80
2,1,Kab. Jembrana,40,37,20% Tinggi,124,2024,0,Tahun,140371240,140371240,39.63
3,2,Kab. Tabanan,40,35,40% Rendah,124,2024,0,Tahun,240351240,240351240,21.80
4,2,Kab. Tabanan,40,36,40% Sedang,124,2024,0,Tahun,240361240,240361240,39.13


In [160]:
# proses cleaning
data40 = data40[['verval_val','vervar_label','tahun_label','content_label','turvar_label']]
data40 = clean_tag_html(data40)
data40_tb = data40.rename(columns={
    'vervar_label' : 'kabupaten_kota',
    'tahun_label'  : 'tahun',
    'content_label': 'pendapatan',
    'turvar_label':'kelompok'
})

data40_tb["kelompok_pendapatan"] = data40_tb["tahun"].astype(str) + "_" + data40_tb["kelompok"].astype(str)
data40_tb = data40_tb.pivot_table(
    index=['verval_val','kabupaten_kota'],
    columns=['kelompok_pendapatan'],
    values=['pendapatan']
).reset_index()

data40_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data40_tb.columns.values
]
data40_tb.head()

,verval_val,kabupaten_kota,pendapatan_2015_20% Tinggi,pendapatan_2015_40% Rendah,pendapatan_2015_40% Sedang,pendapatan_2016_20% Tinggi,pendapatan_2016_40% Rendah,pendapatan_2016_40% Sedang,pendapatan_2017_20% Tinggi,pendapatan_2017_40% Rendah,...,pendapatan_2021_40% Sedang,pendapatan_2022_20% Tinggi,pendapatan_2022_40% Rendah,pendapatan_2022_40% Sedang,pendapatan_2023_20% Tinggi,pendapatan_2023_40% Rendah,pendapatan_2023_40% Sedang,pendapatan_2024_20% Tinggi,pendapatan_2024_40% Rendah,pendapatan_2024_40% Sedang
0,1,Kab. Jembrana,39.16,21.03,39.81,45.04,19.40,35.55,40.05,20.77,...,37.97,42.39,21.46,36.15,42.95,21.92,35.13,39.63,23.57,36.80
1,2,Kab. Tabanan,43.88,19.37,36.76,43.07,19.73,37.20,39.09,21.00,...,39.52,38.91,21.67,39.42,43.67,20.04,36.29,39.07,21.80,39.13
2,3,Kab. Badung,39.71,20.70,39.58,40.04,20.54,39.42,39.47,20.29,...,38.43,41.06,21.28,37.66,38.41,23.52,38.07,37.68,22.99,39.33
3,4,Kab. Gianyar,40.62,20.59,38.79,39.01,21.22,39.77,36.24,23.07,...,38.06,40.02,21.60,38.37,41.17,21.33,37.50,37.88,22.32,39.80
4,5,Kab. Klungkung,45.82,19.54,34.64,44.48,18.99,36.52,46.05,18.92,...,36.87,43.27,19.39,37.35,42.37,20.19,37.44,40.71,22.35,36.95


In [161]:
data40_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (300) Rata-rata Konsumsi Kalori per Kapita Sehari Menurut Kelompok Bahan Makanan Provinsi Bali

In [184]:
varId = 300
data300=getDataByVarId(domain='5100',varId=varId)
data300.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Padi-padian,300,164,Maret,124,2024,0,Tahun,13001641240,13001641240,938.12
2,2,Umbi-umbian,300,164,Maret,124,2024,0,Tahun,23001641240,23001641240,28.92
4,3,Ikan,300,164,Maret,124,2024,0,Tahun,33001641240,33001641240,38.25
6,4,Daging,300,164,Maret,124,2024,0,Tahun,43001641240,43001641240,158.97
8,5,Telur dan Susu,300,164,Maret,124,2024,0,Tahun,53001641240,53001641240,50.00


In [185]:
# proses cleaning
data300 = data300[['verval_val','vervar_label','tahun_label','content_label','turvar_label']]
data300 = clean_tag_html(data300)
data300_tb = data300.rename(columns={
    'vervar_label' : 'kelompok_makanan',
    'tahun_label'  : 'tahun',
    'content_label': 'kalori',
    'turvar_label':'bulan'
})

data300_tb["periode"] = data300_tb["tahun"].astype(str) + "_" + data300_tb["bulan"].astype(str)
data300_tb = data300_tb.pivot_table(
    index=['verval_val','kelompok_makanan'],
    columns=['periode'],
    values=['kalori']
).reset_index()

data300_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data300_tb.columns.values
]
data300_tb.head()

,verval_val,kelompok_makanan,kalori_2015_Maret,kalori_2015_September,kalori_2016_Maret,kalori_2016_September,kalori_2017_Maret,kalori_2017_September,kalori_2018_Maret,kalori_2018_September,kalori_2019_Maret,kalori_2019_September,kalori_2020_Maret,kalori_2021_Maret,kalori_2021_September,kalori_2022_Maret,kalori_2022_September,kalori_2023_Maret,kalori_2024_Maret
0,1,Padi-padian,995.50,996.43,1025.36,940.31,941.44,904.94,925.10,942.35,910.21,896.65,899.29,958.89,974.33,967.41,970.64,961.09,938.12
1,2,Umbi-umbian,26.93,33.83,21.07,20.40,32.89,27.49,26.76,34.84,28.91,28.29,26.73,39.16,41.14,36.10,37.78,39.13,28.92
2,3,Ikan,36.67,34.04,34.41,28.31,33.57,32.18,34.10,38.03,37.52,36.69,36.09,37.31,37.85,35.23,36.52,38.81,38.25
3,4,Daging,82.71,78.13,108.76,234.40,95.07,86.34,97.48,98.29,104.03,95.02,129.78,107.37,87.79,102.27,93.42,110.39,158.97
4,5,Telur dan Susu,59.17,50.88,57.58,50.09,50.74,44.24,52.22,49.52,50.44,50.60,52.95,48.09,49.45,44.53,43.84,48.50,50.00


In [187]:
data300_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (301) Rata-rata Konsumsi Protein per Kapita Sehari Menurut Kelompok Bahan Makanan Provinsi Bali

In [168]:
varId = 301
data301=getDataByVarId(domain='5100',varId=varId)
data301.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Padi-padian,301,164,Maret,124,2024,0,Tahun,13011641240,13011641240,21.99
2,2,Umbi-umbian,301,164,Maret,124,2024,0,Tahun,23011641240,23011641240,0.34
4,3,Ikan,301,164,Maret,124,2024,0,Tahun,33011641240,33011641240,7.10
6,4,Daging,301,164,Maret,124,2024,0,Tahun,43011641240,43011641240,8.04
8,5,Telur dan Susu,301,164,Maret,124,2024,0,Tahun,53011641240,53011641240,3.10


In [170]:
# proses cleaning
data301 = data301[['verval_val','vervar_label','tahun_label','content_label','turvar_label']]
data301 = clean_tag_html(data301)
data301_tb = data301.rename(columns={
    'vervar_label' : 'kelompok_makanan',
    'tahun_label'  : 'tahun',
    'content_label': 'protein',
    'turvar_label':'bulan'
})

data301_tb["periode"] = data301_tb["tahun"].astype(str) + "_" + data301_tb["bulan"].astype(str)
data301_tb = data301_tb.pivot_table(
    index=['verval_val','kelompok_makanan'],
    columns=['periode'],
    values=['protein']
).reset_index()

data301_tb.columns = ['_'.join([c for c in col if c]).strip() if isinstance(col, tuple) else col
    for col in data301_tb.columns.values
]
data301_tb.head()

,verval_val,kelompok_makanan,protein_2015_Maret,protein_2015_September,protein_2016_Maret,protein_2016_September,protein_2017_Maret,protein_2017_September,protein_2018_Maret,protein_2018_September,protein_2019_Maret,protein_2019_September,protein_2020_Maret,protein_2021_Maret,protein_2021_September,protein_2022_Maret,protein_2022_September,protein_2023_Maret,protein_2024_Maret
0,1,Padi-padian,23.35,23.34,24.03,21.97,22.05,21.22,21.69,22.09,21.33,21.02,21.08,22.49,22.85,22.67,22.74,22.54,21.99
1,2,Umbi-umbian,0.29,0.35,0.24,0.21,0.33,0.27,0.28,0.37,0.32,0.31,0.30,0.40,0.42,0.39,0.39,0.43,0.34
2,3,Ikan,6.21,5.77,5.83,4.88,5.85,5.77,5.98,6.71,6.60,6.45,6.32,6.56,6.59,6.47,6.77,7.21,7.10
3,4,Daging,4.50,4.23,5.57,10.87,5.37,4.92,5.46,5.47,5.67,5.41,6.92,6.26,5.29,5.81,5.25,6.26,8.04
4,5,Telur dan Susu,3.52,3.07,3.41,3.09,3.17,2.75,3.21,3.09,3.14,3.12,3.31,3.13,3.23,2.82,2.70,3.04,3.10


In [173]:
data301_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (303) Rata-rata Pengeluaran per Kapita Sebulan Menurut Kelompok Bahan Makanan Provinsi Bali

In [176]:
varId = 303
data303=getDataByVarId(domain='5100',varId=varId)
data303.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Padi-padian,303,0,Tidak ada,123,2023,0,Tahun,130301230,130301230,94632
1,2,Umbi-umbian,303,0,Tidak ada,123,2023,0,Tahun,230301230,230301230,7386
2,3,Ikan,303,0,Tidak ada,123,2023,0,Tahun,330301230,330301230,43118
3,4,Daging,303,0,Tidak ada,123,2023,0,Tahun,430301230,430301230,46950
4,5,Telur dan Susu,303,0,Tidak ada,123,2023,0,Tahun,530301230,530301230,37585


In [177]:
#proses cleaning
data303=data303[['verval_val','vervar_label','tahun_label','content_label']]
#proses rename colomn
data303_tb=data303.rename(columns={'vervar_label' : 'kelompok_makanan','tahun_label' : 'tahun','content_label' : 'pengeluaran'})
#pivoting tabel
data303_tb=data303_tb.pivot_table(index=['verval_val','kelompok_makanan'],columns=['tahun'],values=['pengeluaran'])
#multi index jadikan 1
data303_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data303_tb.columns.values]
data303_tb=data303_tb.reset_index()
data303_tb.head()

,verval_val,kelompok_makanan,pengeluaran_2014,pengeluaran_2015,pengeluaran_2016,pengeluaran_2017,pengeluaran_2018,pengeluaran_2019,pengeluaran_2020,pengeluaran_2021,pengeluaran_2022,pengeluaran_2023
0,1,Padi-padian,67906.0,73055.0,68868.0,67412.0,73029.0,72520.0,77831.0,86583.0,85611.0,94632.0
1,2,Umbi-umbian,2405.0,3112.0,2860.0,4231.0,3750.0,4764.0,4852.0,6588.0,6341.0,7386.0
2,3,Ikan,23314.0,22190.0,21607.0,27140.0,29088.0,33929.0,34073.0,36331.0,34042.0,43118.0
3,4,Daging,26381.0,23935.0,32206.0,33517.0,33586.0,39399.0,50218.0,44372.0,42753.0,46950.0
4,5,Telur dan Susu,27699.0,27234.0,27796.0,30974.0,32453.0,31788.0,37033.0,34879.0,31163.0,37585.0


In [178]:
data303_tb.to_excel(f'output/data_{varId}.xlsx', index=False)

### (305) Rata-rata Pengeluaran per Kapita Sebulan Menurut Kelompok Bukan Makanan Provinsi Bali

In [188]:
varId = 305
data305=getDataByVarId(domain='5100',varId=varId)
data305.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Sewa, kontrak, perkiraan sewa rumah",305,0,Tidak ada,123,2023,0,Tahun,130501230,130501230,275691
1,2,Pemeliharaan dan perbaikan ringan rumah,305,0,Tidak ada,123,2023,0,Tahun,230501230,230501230,6863
2,3,"Listrik, air, gas, minyak tanah, kayu bakar",305,0,Tidak ada,123,2023,0,Tahun,330501230,330501230,89536
3,4,"Rekening telepon rumah, pulsa HP, telepon umum...",305,0,Tidak ada,123,2023,0,Tahun,430501230,430501230,62676
4,5,"Sabun mandi/cuci, kosmetik, perawatan rambut/m...",305,0,Tidak ada,123,2023,0,Tahun,530501230,530501230,74123


In [189]:
#proses cleaning
data305=data305[['verval_val','vervar_label','tahun_label','content_label']]
#proses rename colomn
data305_tb=data305.rename(columns={'vervar_label' : 'kelompok_pengeluaran','tahun_label' : 'tahun','content_label' : 'pengeluaran'})
#pivoting tabel
data305_tb=data305_tb.pivot_table(index=['verval_val','kelompok_pengeluaran'],columns=['tahun'],values=['pengeluaran'])
#multi index jadikan 1
data305_tb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col
                   for col in data305_tb.columns.values]
data305_tb=data305_tb.reset_index()
data305_tb.head()

,verval_val,kelompok_pengeluaran,pengeluaran_2014,pengeluaran_2015,pengeluaran_2016,pengeluaran_2017,pengeluaran_2018,pengeluaran_2019,pengeluaran_2020,pengeluaran_2021,pengeluaran_2022,pengeluaran_2023
0,1,"Sewa, kontrak, perkiraan sewa rumah",167824.0,160052.0,164746.0,209459.0,195869.0,194125.0,215160.0,246810.0,261797.0,275691.0
1,2,Pemeliharaan dan perbaikan ringan rumah,4705.0,5749.0,2691.0,7335.0,9375.0,5134.0,6515.0,4493.0,4343.0,6863.0
2,3,"Listrik, air, gas, minyak tanah, kayu bakar",49354.0,51036.0,55354.0,60319.0,66441.0,69532.0,70073.0,77056.0,77375.0,89536.0
3,4,"Rekening telepon rumah, pulsa HP, telepon umum...",30800.0,34465.0,37211.0,38139.0,43562.0,44865.0,46794.0,52616.0,52642.0,62676.0
4,5,"Sabun mandi/cuci, kosmetik, perawatan rambut/m...",28856.0,36747.0,42411.0,35807.0,46867.0,51004.0,55814.0,58084.0,63461.0,74123.0


In [190]:
data305_tb.to_excel(f'output/data_{varId}.xlsx', index=False)